In [1]:
import Bio # That's biopython 
from Bio import SeqIO # To read the sequences from the fasta files 
from Bio.Seq import Seq # To work with sequences as an object 
from Bio.SeqRecord import SeqRecord
from Bio import motifs # motif library
from Bio.Alphabet import IUPAC
# Pandas' data structure libraries 
import pandas as pd 
import numpy as np


In [2]:
#  test sequences  
Seq1 = Seq("ATGCCTATG", IUPAC.IUPACUnambiguousDNA())  
print (Seq1)
Seq2 = Seq("ATGCCCCCCC", IUPAC.IUPACUnambiguousDNA())
print (Seq2)
Seq3 = Seq("ATGCCCCCCGATG", IUPAC.IUPACUnambiguousDNA()) 
print (Seq3)
test_seq = Seq("TATGATGTAGTATAATATAATTATAA", IUPAC.IUPACUnambiguousDNA())
print test_seq
test_Kozak = Seq("CAAACAAAATG", IUPAC.IUPACUnambiguousDNA())
print test_Kozak
test_antiKozak = Seq("GGGGGTTTTCT", IUPAC.IUPACUnambiguousDNA())
print test_antiKozak

ATGCCTATG
ATGCCCCCCC
ATGCCCCCCGATG
TATGATGTAGTATAATATAATTATAA
CAAACAAAATG
GGGGGTTTTCT


In [3]:
# Create a new object motif "ATG"
motifATG = motifs.create( [ Seq("ATG") ] )
print(motifATG)

ATG



In [4]:
#Create a new motif using kozak PFM distribution backround 
with open("PWMs/Kozak_PFM.txt") as handle: 
        KozakMotif = motifs.read(handle, 'pfm')

print (KozakMotif)

TF name	None
Matrix ID	None
Matrix:
        0      1      2      3      4      5      6      7      8      9     10
A:  60.00  90.00  70.00  80.00  80.00 160.00 110.00 110.00 200.00   0.00   0.00
C:  70.00  60.00  50.00  50.00 100.00  10.00  60.00  60.00   0.00   0.00   0.00
G:  10.00  10.00  30.00  20.00   5.00  25.00  15.00  15.00   0.00   0.00 200.00
T:  60.00  40.00  50.00  50.00  15.00   5.00  15.00  15.00   0.00 200.00   0.00





In [5]:
# From KozakMotif to PWM motif, pseudocounts = ?  
pwmKozak = KozakMotif.counts.normalize(pseudocounts=0.5)
print(pwmKozak)

        0      1      2      3      4      5      6      7      8      9     10
A:   0.30   0.45   0.35   0.40   0.40   0.79   0.55   0.55   0.99   0.00   0.00
C:   0.35   0.30   0.25   0.25   0.50   0.05   0.30   0.30   0.00   0.00   0.00
G:   0.05   0.05   0.15   0.10   0.03   0.13   0.08   0.08   0.00   0.00   0.99
T:   0.30   0.20   0.25   0.25   0.08   0.03   0.08   0.08   0.00   0.99   0.00



In [6]:
# From PwmMotif to pssmMotif using log_odds 

pssmKozak = pwmKozak.log_odds()
print (pssmKozak)

        0      1      2      3      4      5      6      7      8      9     10
A:   0.26   0.84   0.48   0.67   0.67   1.67   1.13   1.13   1.99  -6.66  -6.66
C:   0.48   0.26   0.00   0.00   0.99  -2.27   0.26   0.26  -6.66  -6.66  -6.66
G:  -2.27  -2.27  -0.73  -1.30  -3.20  -0.99  -1.70  -1.70  -6.66  -6.66   1.99
T:   0.26  -0.32   0.00   0.00  -1.70  -3.20  -1.70  -1.70  -6.66   1.99  -6.66



In [7]:
# Normalized score function returns value between 0 and 1, like Bioconductor
def calc_norm_score (pssm, seq) :
    # import pdb; pdb.set_trace()
    raw_score = pssm.calculate(seq)
    score = ( raw_score - pssm.min ) / (pssm.max - pssm.min)
    return ( score )

print calc_norm_score( pssmKozak, test_seq ) # should be an array of numbers between 0 and 1
print calc_norm_score( pssmKozak, test_Kozak ) # should be 1
print calc_norm_score( pssmKozak, test_antiKozak ) # should be 0

[0.3798483  0.4352925  0.15832752 0.553093   0.29560044 0.33615193
 0.59644955 0.33142576 0.6381159  0.3224843  0.47655964 0.68698126
 0.52971977 0.35268602 0.66425353 0.30258524]
1.0000000052353402
2.3204725370384962e-08


In [8]:
def get_pos_frame (sequenceID, sequence, motif):
    # Calculate for a given sequence the ATGs, their position and frames 
    index = 0
    position = 0
    frame = 0
    score = 0
    sequence.alphabet = IUPAC.unambiguous_dna
    df = pd.DataFrame(columns = ['Sequence','ATGNumber','position','frame'])
    for pos, seq in  motif.instances.search(sequence):
        index = index + 1 
        ATGNumber = index
        position = pos
        frame = pos % 3
        #import pdb; pdb.set_trace()
        df = df.append({'Sequence': sequenceID,
                'ATGNumber':ATGNumber,
                'position':position,
                'frame':frame}, ignore_index=True)
               
    #if (index == 1):
     #   print("No other alternative frames!")  
    #frame = -1
    #position = -1
    #ATGNumber = index
    return (df)

In [9]:
# test get_pos_frame with one sequence at a time
print get_pos_frame("Seq1",Seq1,motifATG)
print get_pos_frame("Seq2",Seq2,motifATG)
print get_pos_frame("Seq3",Seq3,motifATG)

  Sequence ATGNumber position frame
0     Seq1         1        0     0
1     Seq1         2        6     0
  Sequence ATGNumber position frame
0     Seq2         1        0     0
  Sequence ATGNumber position frame
0     Seq3         1        0     0
1     Seq3         2       10     1


In [10]:
# The function OutputPosFrame takes a Fasta file containing sequences and writes a txt file containing the position of the 
#alternative translation initiation and the frame  
def output_pos_frame (input_file, output_file, motif=motifATG): 
    dfresult = pd.DataFrame(columns = ['Sequence','ATGNumber','position','frame'])
    for seq_record in SeqIO.parse(input_file, "fasta"):
        dfbit = get_pos_frame(seq_record.id, seq_record.seq, motifATG)
        dfresult = dfresult.append(dfbit, ignore_index = True)
    dfresult.to_csv (output_file, encoding ='utf-8', index = False)
    return ( dfresult )


In [11]:
# test the output_pos_frame function with 10 sequences from H99
output_pos_frame ('Sequences/test_10_H99_CDS.fa','Output/test_10_ATG_pos_frame_H99_CDS.csv')

,Sequence,ATGNumber,position,frame
0,CNAG_04548.t01,1,0,0
1,CNAG_04548.t01,2,28,1
2,CNAG_04548.t01,3,168,0
3,CNAG_04548.t01,4,207,0
4,CNAG_04548.t01,5,243,0
5,CNAG_04548.t01,6,274,1
6,CNAG_04548.t01,7,295,1
7,CNAG_04548.t01,8,337,1
8,CNAG_04548.t01,9,347,2
9,CNAG_04548.t01,10,471,0


In [ ]:
output_pos_frame ('Sequences/H99_CDS.fa','Output/ATG_pos_frame_H99_CDS.csv')

In [ ]:
# output_pos_frame ("Homologus fungal/Candida_albicans_sc5314.Cand_albi_SC5314_V4.cds.all.fa")